# Data labeling and human-in-the-loop pipelines with Amazon Augmented AI (A2I)
In this lab you will create your own human workforce, a human task UI, and then define the human review workflow to perform data labeling. You will make the original predictions of the labels with the custom ML model, and then create a human loop if the probability scores are lower than the preset threshold. After the completion of the human loop tasks, you will review the results and prepare data for re-training.

In [3]:
# please ignore warning messages during the installation
!pip install --disable-pip-version-check -q sagemaker==2.35.0

import boto3, sagemaker, pandas as pd, botocore
from pprint import pprint

config = botocore.config.Config(user_agent_extra='dlai-pds/c3/w3')

# low-level service client of the boto3 session
sm = boto3.client(service_name='sagemaker', config=config)

sm_runtime = boto3.client('sagemaker-runtime', config=config)

sess = sagemaker.Session(sagemaker_client=sm, sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

s3 = boto3.Session().client(service_name='s3', config=config)
cognito_idp = boto3.Session().client(service_name='cognito-idp', config=config)
a2i = boto3.Session().client(service_name='sagemaker-a2i-runtime', config=config)

## 1. Set up Amazon Cognito user pool and define human workforce
<a name='c3w3-1.'></a>
<img src="./c3w3/images/human-loop-workflow-1-workforce.png" width="40%" align="center">
The first step in the creation of the human-in-the-loop pipeline will be to create your own private workforce.
Amazon Cognito provides authentication, authorization, and user management for apps. This enables your workers to sign in directly to the labeling UI with a username and password. 
You will construct an Amazon Cognito user pool, setting up its client, domain, and group. Then you'll create a SageMaker workforce, linking it to the Cognito user pool. Followed by the creation of a SageMaker workteam, linking it to the Cognito user pool and group. And finally, you will create a pool user and add it to the group.

In [ ]:
#construct the user pool and user pool client names.
import time
timestamp = int(time.time())

user_pool_name = 'groundtruth-user-pool-{}'.format(timestamp)
user_pool_client_name = 'groundtruth-user-pool-client-{}'.format(timestamp)

print("Amazon Cognito user pool name: {}".format(user_pool_name))
print("Amazon Cognito user pool client name: {}".format(user_pool_client_name))

#Create Amazon Cognito user pool:
create_user_pool_response = cognito_idp.create_user_pool(PoolName=user_pool_name)
#Pull the Amazon Cognito user pool name from its description
print(create_user_pool_response['UserPool'].keys())
user_pool_name = create_user_pool_response["UserPool"]["Name"] 
print('Amazon Cognito user pool name: {}'.format(user_pool_name))
user_pool_id = create_user_pool_response['UserPool']['Id']
print("Amazon Cognito user pool ID: {}".format(user_pool_id))

In [ ]:
#set up the Amazon Cognito user pool client for the created above user pool.
#The Amazon Cognito user pool client implements an open standard for authorization framework, OAuth. The standard enables apps to obtain limited access (scopes) to a user’s data without giving away a user’s password. It decouples authentication from authorization and supports multiple use cases addressing different device capabilities.
create_user_pool_client_response = cognito_idp.create_user_pool_client( # Replace None
    UserPoolId=user_pool_id, 
    ClientName=user_pool_name, 
    GenerateSecret=True, # boolean to specify whether you want to generate a secret
    # a list of provider names for the identity providers that are supported on this client, e.g. Cognito, Facebook, Google
    SupportedIdentityProviders=['COGNITO' ],
    # a list of the allowed OAuth flows, e.g. code, implicit, client_credentials
    AllowedOAuthFlows=['code','implicit'],
    # a list of the allowed OAuth scopes, e.g. phone, email, openid, and profile
    AllowedOAuthScopes=['email','openid','profile'],
    # a list of allowed redirect (callback) URLs for the identity providers
    CallbackURLs=['https://datascienceonaws.com', ],
    # set to true if the client is allowed to follow the OAuth protocol when interacting with Cognito user pools
    AllowedOAuthFlowsUserPoolClient=True)

client_id = create_user_pool_client_response['UserPoolClient']['ClientId']
print('Amazon Cognito user pool client ID: {}'.format(client_id))

#Set up the Amazon Cognito user pool domain for the constructed user pool:
user_pool_domain_name = 'groundtruth-user-pool-domain-{}'.format(timestamp)
try:
    cognito_idp.create_user_pool_domain(UserPoolId=user_pool_id, Domain=user_pool_domain_name )
    print("Created Amazon Cognito user pool domain: {}".format(user_pool_domain_name))
except:
    print("Amazon Cognito user pool domain {} already exists".format(user_pool_domain_name))
    
#check if the Amazon Cognito user group already exists:
def check_user_pool_group_existence(user_pool_id, user_pool_group_name):  
    for group in cognito_idp.list_groups(UserPoolId=user_pool_id)['Groups']:
        if user_pool_group_name == group['GroupName']:
            return True
    return False

#Set up Amazon Cognito user group:
user_pool_group_name = 'groundtruth-user-pool-group-{}'.format(timestamp)
if not check_user_pool_group_existence(user_pool_id, user_pool_group_name):
    cognito_idp.create_group( UserPoolId=user_pool_id, GroupName=user_pool_group_name )
    print("Created Amazon Cognito user group: {}".format(user_pool_group_name))
else:
    print("Amazon Cognito user group {} already exists".format(user_pool_group_name))
    
#Create a workforce: check if the workforce already exists:
def check_workforce_existence(workforce_name):  
    for workforce in sm.list_workforces()['Workforces']:
        if workforce_name == workforce['WorkforceName']:
            return True
        else:
            for workteam in sm.list_workteams()['Workteams']:
                sm.delete_workteam(WorkteamName=workteam['WorkteamName'])
            sm.delete_workforce(WorkforceName=workforce['WorkforceName'])
    return False
#Create a workforce
workforce_name = 'groundtruth-workforce-name-{}'.format(timestamp)
if not check_workforce_existence(workforce_name):
    create_workforce_response = sm.create_workforce(WorkforceName=workforce_name,
        CognitoConfig={'UserPool': user_pool_id, 'ClientId': client_id})
    print("Workforce name: {}".format(workforce_name))
    pprint(create_workforce_response)
else:
    print("Workforce {} already exists".format(workforce_name))

#to get the information about the workforce
describe_workforce_response = sm.describe_workforce(WorkforceName=workforce_name)
describe_workforce_response

In [ ]:
#wait till above done
#Create a workteam: check if the workteam already exists
def check_workteam_existence(workteam_name):  
    if sm.list_workteams()['Workteams']:
        for workteam in sm.list_workteams()['Workteams']:
            if workteam_name == workteam['WorkteamName']:
                return True
    else:
        time.sleep(60)
        return False
    return False
#Create a workteam:
workteam_name = 'groundtruth-workteam-{}'.format(timestamp)

if not check_workteam_existence(workteam_name):
    create_workteam_response = sm.create_workteam(
        Description='groundtruth workteam', WorkforceName=workforce_name, WorkteamName=workteam_name,
        # objects that identify the workers that make up the work team
        MemberDefinitions=[{'CognitoMemberDefinition': {'UserPool': user_pool_id, 'ClientId': client_id, 'UserGroup': user_pool_group_name}}])
    pprint(create_workteam_response)
else:
    print("Workteam {} already exists".format(workteam_name))

#to get information about the workteam:
describe_workteam_response = sm.describe_workteam(WorkteamName=workteam_name)
describe_workteam_response
#pull the workteam ARN either from create_workteam_response or describe_workteam_response:
workteam_arn = describe_workteam_response['Workteam']['WorkteamArn']
workteam_arn

#Review the created workteam in the AWS console:
from IPython.core.display import display, HTML
display(HTML('<b>Review <a target="blank" href="https://{}.console.aws.amazon.com/sagemaker/groundtruth?region={}#/labeling-workforces/private-details/{}">workteam</a></b>'.format(region, region, workteam_name)))

#Create an Amazon Cognito user and add the user to the group: check if the Amazon Cognito user already exists
def check_user_existence(user_pool_id, user_name):  
    for user in cognito_idp.list_users(UserPoolId=user_pool_id)['Users']:
        if user_name == user['Username']:
            return True
    return False

#Create a user:
user_name = 'user-{}'.format(timestamp)
temporary_password = 'Password@420'

if not check_user_existence(user_pool_id, user_name):
    create_user_response=cognito_idp.admin_create_user(
        Username=user_name, UserPoolId=user_pool_id, TemporaryPassword=temporary_password,MessageAction='SUPPRESS') # suppress sending the invitation message to a user that already exists 
    pprint(create_user_response)
else:
    print("Amazon Cognito user {} already exists".format(user_name))

#Add the user into the Amazon Cognito user group:
cognito_idp.admin_add_user_to_group(UserPoolId=user_pool_id, Username=user_name, GroupName=user_pool_group_name)

<a name='c3w3-2.'></a>
## 2. Create Human Task UI
Create a Human Task UI resource, using a worker task UI template.  This template will be rendered to the human workers whenever human interaction is required.
Below there is a simple demo template provided, that is compatible with the current use case of classifying product reviews into the three sentiment classes. For other pre-built UIs (there are 70+), check: https://github.com/aws-samples/amazon-a2i-sample-task-uis

In [21]:
template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
    <crowd-classifier name="sentiment"
                      categories="['-1', '0', '1']"
                      initial-value="{{ task.input.initialValue }}"
                      header="Classify Reviews into Sentiment:  -1 (negative), 0 (neutral), and 1 (positive)">
      
        <classification-target>
            {{ task.input.taskObject }}
        </classification-target>
      
        <full-instructions header="Classify reviews into sentiment:  -1 (negative), 0 (neutral), and 1 (positive)">
            <p><strong>1</strong>: joy, excitement, delight</p>       
            <p><strong>0</strong>: neither positive or negative, such as stating a fact</p>
            <p><strong>-1</strong>: anger, sarcasm, anxiety</p>
        </full-instructions>

        <short-instructions>
            Classify reviews into sentiment:  -1 (negative), 0 (neutral), and 1 (positive)
        </short-instructions>
    </crowd-classifier>
</crowd-form>
"""

#Create a human task UI resource:
task_ui_name = 'ui-{}'.format(timestamp)# this value is unique per account and region. You can also provide your own value here.
human_task_ui_response = sm.create_human_task_ui(HumanTaskUiName=task_ui_name, UiTemplate={"Content": template})
human_task_ui_response

#Pull the ARN of the human task UI:
human_task_ui_arn = human_task_ui_response["HumanTaskUiArn"]
print(human_task_ui_arn)

arn:aws:sagemaker:us-east-1:170235698766:human-task-ui/ui-1659019561


## Define human review workflow
In this section, you are going to create a Flow Definition. Flow Definitions allow you to specify:
- The workforce (in fact, it is a workteam) that your tasks will be sent to.
- The instructions that your workforce will receive (worker task template).
- The configuration of your worker tasks, including the number of workers that receive a task and time limits to complete tasks.
- Where your output data will be stored.

Here you are going to use the API, but you can optionally create this workflow definition in the console as well. 
For more details and instructions, see: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html.

In [24]:
#construct the S3 bucket output path:
output_path = 's3://{}/a2i-results-{}'.format(bucket, timestamp)
print(output_path)

#Construct the Flow Definition with the workteam and human task UI in the human loop configurations that you created above:
flow_definition_name = 'fd-{}'.format(timestamp)#this value is unique per account and region

create_workflow_definition_response = sm.create_flow_definition(
    FlowDefinitionName=flow_definition_name, RoleArn=role,
    HumanLoopConfig={        
        "WorkteamArn": workteam_arn, "HumanTaskUiArn": human_task_ui_arn,
        "TaskCount": 1, # the number of workers that receive a task
        "TaskDescription": "Classify Reviews into sentiment:  -1 (negative), 0 (neutral), 1 (positive)",
        "TaskTitle": "Classify Reviews into sentiment:  -1 (negative), 0 (neutral), 1 (positive)",},
    OutputConfig={"S3OutputPath": output_path},)

augmented_ai_flow_definition_arn = create_workflow_definition_response["FlowDefinitionArn"]

#pull information about the Flow Definition with the function and wait for its status value FlowDefinitionStatus to become Active.
for _ in range(60):
    describe_flow_definition_response = sm.describe_flow_definition(FlowDefinitionName=flow_definition_name)
    print(describe_flow_definition_response["FlowDefinitionStatus"])
    if describe_flow_definition_response["FlowDefinitionStatus"] == "Active":
        print("Flow Definition is active")
        break
    time.sleep(2)

s3://sagemaker-us-east-1-170235698766/a2i-results-1659019561
Active
Flow Definition is active


## 4. Start human loop with custom ML model
<img src="./c3w3/images/human-loop-workflow-4-start.png" width="40%" align="center">

Deploy a custom ML model into an endpoint and call it to predict labels for some sample reviews. Check the confidence score for each prediction. If it is smaller than the threshold, engage your workforce for a human review, starting a human loop. Fix the labels by completing the human loop tasks and review the results.

<img src="./c3w3/images/augmented-ai-loop.png" width="60%" align="center">

In [28]:
#Deploy a custom model: Set up a sentiment predictor class to be wrapped later into the PyTorch Model.
#Create a Sentiment Predictor class
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

class SentimentPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__( endpoint_name, sagemaker_session=sagemaker_session, serializer=JSONLinesSerializer, deserializer=JSONLinesDeserializer)
        
#Create a SageMaker model based on the model artifact saved in the S3 bucket:
from sagemaker.pytorch.model import PyTorchModel
pytorch_model_name = 'model-{}'.format(timestamp)

model = PyTorchModel(name=pytorch_model_name,model_data='s3://dlai-practical-data-science/models/ab/variant_a/model.tar.gz',
                     predictor_cls=SentimentPredictor, entry_point='inference.py', source_dir='c3w3/src', framework_version='1.6.0', py_version='py3', role=role)

In [29]:
#Now you will create a SageMaker Endpoint from the model:
#%%time
pytorch_endpoint_name = 'endpoint-{}'.format(timestamp)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.large', endpoint_name=pytorch_endpoint_name)

#review the endpoint in the AWS console:
from IPython.core.display import display, HTML
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, pytorch_endpoint_name)))

----------!

In [ ]:
#Start the human loop
#create a list of sample reviews:
reviews = ["I enjoy this product", "I am unhappy with this product", "It is okay", "sometimes it works"]

#Now you can send each of the sample reviews to the model via the `predictor.predict()` API call. Note that you need to pass the reviews in the JSON format that model expects as input. Then, you parse the model's response to obtain the predicted label and the confidence score. After that, you check the condition for when you want to engage a human for review. You can check whether the returned confidence score is under the defined threshold of 90%, which would mean that you would want to start the human loop with the predicted label and the review as inputs. Finally, you start the human loop passing the input content and Flow Definition defined above.
import json
human_loops_started = []
CONFIDENCE_SCORE_THRESHOLD = 0.90

for review in reviews:
    inputs = [{"features": [review]},]
    response = predictor.predict(inputs)
    print(response)
    prediction = response[0]['predicted_label']
    confidence_score = response[0]['probability']
    print('Checking prediction confidence {} for sample review: "{}"'.format(confidence_score, review))
    
    if confidence_score < CONFIDENCE_SCORE_THRESHOLD: # condition for when you want to engage a human for review
        human_loop_name = str(time.time()).replace('.', '-') # using milliseconds
        input_content = {"initialValue": None, "taskObject": None }
        start_loop_response = a2i.start_human_loop(HumanLoopName=human_loop_name,
            FlowDefinitionArn=augmented_ai_flow_definition_arn, HumanLoopInput={"InputContent": json.dumps(input_content)},)        
        human_loops_started.append(human_loop_name)
        print(f"Confidence score of {confidence_score * 100}% for prediction of {prediction} is less than the threshold of {CONFIDENCE_SCORE_THRESHOLD * 100}%")
        print(f"*** ==> Starting human loop with name: {human_loop_name}  \n")
    else:
        print(f"Confidence score of {confidence_score * 100}% for star rating of {prediction} is above threshold of {CONFIDENCE_SCORE_THRESHOLD * 100}%")
        print("Human loop not needed. \n")
#Review the results above. Three of the sample reviews with the probability scores lower than the threshold went into the human loop. The original predicted labels are passed together with the review text and will be seen in the task.        

#Check status of the human loop:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f"HumanLoop Name: {human_loop_name}")
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print("")
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

#Complete the human loop tasks:
#Pull labeling UI from the workteam information to get into the human loop tasks in the AWS console:
labeling_ui = sm.describe_workteam(WorkteamName=workteam_name)["Workteam"]["SubDomain"]
print(labeling_ui)

#Navigate to the link below and login with the defined username and password. Complete the human loop following the provided instructions.
from IPython.core.display import display, HTML
display(HTML('Click <a target="blank" href="https://{}"><b>here</b></a> to start labeling with username <b>{}</b> and temporary password <b>{}</b>'.format(labeling_ui, user_name, temporary_password)))

In [ ]:
#Verify that the human loops were completed by the workforce:
#Note: This cell will not complete until you label the data following the instructions above.
import time

completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f"HumanLoop Name: {human_loop_name}")
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print("")
    while resp["HumanLoopStatus"] != "Completed":
        print(f"Waiting for HumanLoop to complete.")
        time.sleep(10)
        resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)
        print(f"Completed!")
        print("")

In [ ]:
#View human labels and prepare the data for re-training
#Once the work is complete, Amazon A2I stores the results in the specified S3 bucket and sends a Cloudwatch Event. Let's check the S3 contents.
import re
from pprint import pprint
fixed_items = []

for resp in completed_human_loops:
    split_string = re.split("s3://" + bucket + "/", resp["HumanLoopOutput"]["OutputS3Uri"])
    output_bucket_key = split_string[1]

    response = s3.get_object(Bucket=bucket, Key=output_bucket_key)
    content = response["Body"].read().decode("utf-8")
    json_output = json.loads(content)
    pprint(json_output)

    input_content = json_output["inputContent"]
    human_answer = json_output["humanAnswers"][0]["answerContent"]
    fixed_item = {"input_content": input_content, "human_answer": human_answer}
    fixed_items.append(fixed_item)

#Now you can prepare the data for re-training:
df_fixed_items = pd.DataFrame(fixed_items)  
df_fixed_items.head()